In [12]:
from pandas import DataFrame, read_csv
import random
import math

In [13]:
random.gauss(0, 0.5)

-0.2788879978918221

In [14]:
paramsToStore = {"Time": [0,1,2,3,0,1,2,3,4,5],
                "TTC": [random.uniform(0, 10) for _ in range(10)],
                "DTO": [random.uniform(0, 20) for _ in range(10)],
                "JERK": [abs(random.gauss(0, 20)) for _ in range(10)],
                "Speed": [random.uniform(0, 15) for _ in range(10)],
                "AngularSpeedX": [random.gauss(0, 0.15) for _ in range(10)],
                "AngularSpeedY": [random.gauss(0, 0.15) for _ in range(10)],
                "AngularSpeedZ": [random.gauss(0, 0.1) for _ in range(10)],
                "COL": [0,0,0,1,0,0,0,0,1,0]}
df = DataFrame(paramsToStore)
# df.set_index("Time", inplace=True)
# df.to_csv(PATH+"/data/testGenerateData.csv", index=False)
df.head(10)

,Time,TTC,DTO,JERK,Speed,AngularSpeedX,AngularSpeedY,AngularSpeedZ,COL
0,0,7.116975,17.002910,13.229286,5.458150,-0.155378,-0.162511,0.051047,0
1,1,7.826149,19.507053,40.566420,4.621862,0.258344,0.065944,0.143950,0
2,2,6.703581,10.688708,40.867080,3.225628,-0.140147,0.106163,0.135206,0
3,3,5.238501,9.576592,14.198564,0.446833,-0.195884,-0.027310,0.010459,1
4,0,6.936956,3.469441,4.447441,11.650245,-0.081614,-0.090311,0.138897,0
5,1,1.251372,6.573449,33.983241,10.600008,0.349623,-0.140615,0.050342,0
6,2,2.738365,13.945958,45.619052,4.352003,-0.121323,0.336551,-0.080230,0
7,3,4.948840,1.179586,31.512473,14.895607,0.068952,0.116351,-0.100193,0
8,4,3.518914,8.442456,17.946443,11.852467,-0.025403,0.185374,0.078078,1
9,5,3.736883,19.770432,15.694971,14.327327,-0.068874,-0.089616,-0.042136,0


In [15]:
l = [0]
l[-3:] = [1]*3
print(l)

[1, 1, 1]


In [16]:
def makeDataUsable(df: DataFrame, pastImportance: int=6, secBeforeCol: int=3) -> DataFrame:
    """
    Make a usable Dataframe from a csv file from generated data.\\
    Creates pastImportance of columns for each prediction feature.\\
    Registers the "col" feature before an actual collision as 1, up to secBeforeCol back in time.
    
    ### Params:
        * df: Dataframe, need to have the columns named "TTC", "DTO", "JERK" and "Speed"ArithmeticError
        * pastImportance: int, needs to be > 0
        * secBeforeCol: int, needs to be > 0
    ### Returns:
        * Dataframe
    """
    colsToUse = ["TTC", "DTO", "JERK", "Speed", "asX", "asY", "asZ"]
    columns = ["Time"]
    columns += [f"{c}{i}" for c in colsToUse for i in range(1, pastImportance+1)]
    columns += ["COL"]
    
    # colsToUse = list(df.columns)
    # columns = colsToUse[0]
    # columns += [f"{c}{i}" for i in range(1, pastImportance+1) for c in colsToUse[1:-1]]
    # columns = colsToUse[-1]
    
    dataDict = {col: [] for col in columns}

    for i, row in df.iterrows():
        if row["Time"] < pastImportance-1:
        # if row[colsToUse[0]] < pastImportance-1:   
            continue

        dataDict["Time"].append(row["Time"])
        # dataDict[colsToUse[0]].append(row[colsToUse[0]])
        for j, k in enumerate(range(i-pastImportance+1, i+1), start=1):
            for c in colsToUse:
            # for c in colsToUse[1:-1]:
                dataDict[f"{c}{j}"].append(df.iloc[k][c])

        dataDict[f"COL"].append(row["COL"])
        if row["COL"] == 1:
            available = int(row["Time"]-pastImportance+2)
            amount = int(secBeforeCol) if available >= secBeforeCol else available
            dataDict["COL"][-amount:] = [1]*amount

    # for key, val in dataDict.items():
    #     print(key, val, len(val))
    return DataFrame(dataDict)

# makeDataUsable(df, 3, 3)
makeDataUsable(read_csv("../../data/testGenerateData.csv"),4, 3)

,Time,TTC1,TTC2,TTC3,TTC4,DTO1,DTO2,DTO3,DTO4,JERK1,...,asX4,asY1,asY2,asY3,asY4,asZ1,asZ2,asZ3,asZ4,COL
0,3.0,0.000,50.000,50.000,50.000,0.0000,100.0000,100.0000,100.0000,0.000,...,-0.003,0.000,-0.000,-0.001,0.000,0.000,0.003,-0.001,-0.003,0.0
1,4.0,50.000,50.000,50.000,50.000,100.0000,100.0000,100.0000,100.0000,1.879,...,-0.000,-0.000,-0.001,0.000,0.000,0.003,-0.001,-0.003,-0.000,0.0
2,5.0,50.000,50.000,50.000,50.000,100.0000,100.0000,100.0000,100.0000,3.210,...,0.014,-0.001,0.000,0.000,-0.001,-0.001,-0.003,-0.000,0.014,0.0
3,6.0,50.000,50.000,50.000,50.000,100.0000,100.0000,100.0000,100.0000,1.895,...,-0.000,0.000,0.000,-0.001,-0.001,-0.003,-0.000,0.014,0.000,0.0
4,7.0,50.000,50.000,50.000,3.982,100.0000,100.0000,100.0000,79.6022,3.411,...,-0.000,0.000,-0.001,-0.001,0.000,-0.000,0.014,0.000,-0.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,297.0,50.000,50.000,1.292,0.590,17.5372,63.1460,34.8705,13.0495,1.825,...,0.063,0.703,-0.608,0.190,0.107,0.008,-0.017,0.022,0.066,0.0
295,298.0,50.000,1.292,0.590,50.000,63.1460,34.8705,13.0495,18.0880,0.681,...,-0.036,-0.608,0.190,0.107,0.002,-0.017,0.022,0.066,-0.015,0.0
296,299.0,1.292,0.590,50.000,50.000,34.8705,13.0495,18.0880,20.2680,0.420,...,-0.016,0.190,0.107,0.002,0.061,0.022,0.066,-0.015,0.016,0.0
297,300.0,0.590,50.000,50.000,50.000,13.0495,18.0880,20.2680,20.5062,3.161,...,0.081,0.107,0.002,0.061,0.551,0.066,-0.015,0.016,0.011,0.0


In [17]:
data = makeDataUsable(read_csv("../../data/testGenerateData.csv"), 6, 4)
data.head(10)

,Time,TTC1,TTC2,TTC3,TTC4,TTC5,TTC6,DTO1,DTO2,DTO3,...,asY4,asY5,asY6,asZ1,asZ2,asZ3,asZ4,asZ5,asZ6,COL
0,5.0,0.000,50.000,50.000,50.000,50.000,50.000,0.0000,100.0000,100.0000,...,0.000,0.000,-0.001,0.000,0.003,-0.001,-0.003,-0.000,0.014,0.0
1,6.0,50.000,50.000,50.000,50.000,50.000,50.000,100.0000,100.0000,100.0000,...,0.000,-0.001,-0.001,0.003,-0.001,-0.003,-0.000,0.014,0.000,0.0
2,7.0,50.000,50.000,50.000,50.000,50.000,3.982,100.0000,100.0000,100.0000,...,-0.001,-0.001,0.000,-0.001,-0.003,-0.000,0.014,0.000,-0.000,0.0
3,8.0,50.000,50.000,50.000,50.000,3.982,17.512,100.0000,100.0000,100.0000,...,-0.001,0.000,0.000,-0.003,-0.000,0.014,0.000,-0.000,-0.000,0.0
4,9.0,50.000,50.000,50.000,3.982,17.512,1.706,100.0000,100.0000,100.0000,...,0.000,0.000,-0.001,-0.000,0.014,0.000,-0.000,-0.000,0.009,0.0
5,10.0,50.000,50.000,3.982,17.512,1.706,6.172,100.0000,100.0000,79.6022,...,0.000,-0.001,-0.000,0.014,0.000,-0.000,-0.000,0.009,-0.007,0.0
6,11.0,50.000,3.982,17.512,1.706,6.172,3.542,100.0000,79.6022,75.4959,...,-0.001,-0.000,0.000,0.000,-0.000,-0.000,0.009,-0.007,-0.000,0.0
7,12.0,3.982,17.512,1.706,6.172,3.542,2.755,79.6022,75.4959,47.1299,...,-0.000,0.000,0.000,-0.000,-0.000,0.009,-0.007,-0.000,0.000,1.0
8,13.0,17.512,1.706,6.172,3.542,2.755,2.028,75.4959,47.1299,39.2978,...,0.000,0.000,0.000,-0.000,0.009,-0.007,-0.000,0.000,0.000,1.0
9,14.0,1.706,6.172,3.542,2.755,2.028,1.106,47.1299,39.2978,30.8573,...,0.000,0.000,0.000,0.009,-0.007,-0.000,0.000,0.000,-0.000,1.0


In [1]:
from useGeneratedData import makeDataUsable, splitTrainTest, NewPredicter
from pandas import DataFrame, read_csv

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [8]:
data = makeDataUsable(read_csv("../../data/testGenerateData.csv"), 6, 5)
data.drop(columns=data.columns[0], axis=1, inplace=True)
# print(data.head(10))
trainX, trainY, testX, testY = splitTrainTest(data)
print(f"trainX:{trainX.shape}, trainY:{trainY.shape}, testX:{testX.shape}, testY:{testY.shape}")
# print(trainX.head())
print(f"trainX: {type(trainX)}, trainY: {type(trainY)}, testX: {type(testX)}, testY: {type(testY)}")


splitting at 237.
trainX:(237, 42), trainY:(237,), testX:(60, 42), testY:(60,)
trainX: <class 'pandas.core.frame.DataFrame'>, trainY: <class 'pandas.core.series.Series'>, testX: <class 'pandas.core.frame.DataFrame'>, testY: <class 'pandas.core.series.Series'>


In [9]:

# p = NewPredicter.loadModel("xgb_2_582-11-16-201")
# print(p.__dict__)

p = NewPredicter()
trainXpp = p.preProcess(trainX)
trainYpp = trainY.to_numpy()
testXpp = p.preProcess(testX)
testYpp = testY.to_numpy()
print(f"trainX: {type(trainXpp)}, trainY: {type(trainYpp)}, testX: {type(testXpp)}, testY: {type(testYpp)}")
# print(trainXpp)
p.fit(trainXpp, trainYpp)

# p.predict([])

amount of features: 0
Scaler is fitted
trainX: <class 'numpy.ndarray'>, trainY: <class 'numpy.ndarray'>, testX: <class 'numpy.ndarray'>, testY: <class 'numpy.ndarray'>


In [11]:
pred = []
for i in range(len(testX)):
    pred.append(p.predict(testX.iloc[i]))
print(pred)

[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [12]:
p.getScore(pred, testY)

Total: 60, number of collisions: 11
	TN: 43 	| FP: 6 
	FN: 8 	| TP: 3
Accuracy: 0.77
Precision: 0.33
Recall: 0.27
F1: 0.3


[[43, 6], [8, 3]]